In [1]:
import simulate
from PlateSolve3.PlaneWave import platesolve
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from astropy.io import fits
import astropy.units as u
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import scoreatpercentile
from os import walk,path,makedirs,listdir
import shutil
%matplotlib tk

In [2]:
def make_image(ra,dec,bgd,seeing,texp,maxmag,resolution,dirname,filename,kerdir=None):
    params = {'max_mag':maxmag,
          'ra':ra, 'dec': dec,
          'op_mode':'Feed',
          't_exp':texp,
          'bgd':bgd,
          'seeing':seeing,
          'temp':-15,
          'read_rms':3,
          'resolution':resolution,
          'kernel_dir':kerdir
          }
    image = simulate.simulate_image(params)
    filepath = path.join('.','images',dirname,filename) + '.fits'
    simulate.save_as_fits(image,params,filepath)

def create_images_uniformly(N,bgd,seeing,dirname):
    icdf = lambda p: np.rad2deg(np.arccos(1 - 2*p))
    dec_arr = icdf(np.random.random(N)) - 90
    ra_arr = np.random.uniform(0,360,N)
    i_arr = np.arange(1,N+1,1)
    for ra,dec,i in tqdm(zip(ra_arr,dec_arr,i_arr),total = len(i_arr)):
        make_image(ra,dec,bgd,seeing,1,15,dirname,f'{int(i)}')

def create_precision_images(N,bgd,seeing,resolution):
    base_dir = 'prec'
    p = simulate.plate_scale_arcsec_px.value/3600

    icdf = lambda p: np.rad2deg(np.arccos(1 - 2*p))
    center_dec = icdf(np.random.random(N)) - 90
    center_ra = np.random.uniform(0,360,N)
    i_arr = np.arange(1,N+1,1)
    for ra,dec,i in tqdm(zip(center_ra,center_dec,i_arr),total = len(i_arr)):
        dirname = path.join(base_dir,f'{int(i)}')
        path_check = path.join('.','images',dirname)
        if not path.exists(path_check):
            makedirs(path_check)

        make_image(ra,dec,bgd,seeing,1,15,resolution,dirname,'cntr')

        delta_ra = p * np.cos(dec)
        delta_dec = p

        shift_ra = np.random.uniform(-4,4) * delta_ra
        shift_dec = np.random.uniform(-4,4) * delta_dec
        ra2 = ra + shift_ra
        dec2 = dec + shift_dec
        make_image(ra2,dec2,bgd,seeing,1,15,resolution,dirname,'shift')
        
def rand_solve(l,h,t_exp):
    i = np.random.randint(low = l,high = h)
    platesolve.platesolve(f'./images/exp{int(t_exp)}/{i}.fits',simulate.plate_scale_arcsec_px.value)

def stats_solve(dirpath):
    filenames = next(walk(dirpath), (None, None, []))[2]  # [] if no file
    n = len(filenames)
    ra_img = np.zeros(n)
    dec_img = np.zeros(n)
    ra_ps = np.full(n, np.nan)
    dec_ps = np.full(n, np.nan)
    for i in tqdm(range(n)):
        hdu = fits.open(path.join(dirpath,filenames[i]))
        ra_img[i] = hdu[0].header['RA']
        dec_img[i] = hdu[0].header['DEC']
        hdu.close()
        try:
            ps = platesolve.platesolve(path.join(dirpath,filenames[i]),simulate.plate_scale_arcsec_px.value)
            ra_ps[i] = ps['ra_j2000_hours'] * 15
            dec_ps[i] = ps['dec_j2000_degrees']
        except:
            pass
    return ra_img,dec_img,ra_ps,dec_ps

def precision_solve(dirpath):
    ra_vec = []
    dec_vec = []
    ra_shift_vec = []
    ra_corr_vec = []
    dec_shift_vec = []
    dec_corr_vec = []
    dirs = listdir(dirpath)

    with tqdm(total = len(dirs)) as pbar:
        for cur_dir in dirs:
            cur_path = path.join(dirpath,cur_dir)
            filenames = next(walk(cur_path), (None, None, []))[2]  # [] if no file
            n = len(filenames)

            cur_ra = []
            cur_ra_ps = []
            cur_dec = []
            cur_dec_ps = []
            ra0 = 0
            ra0_ps = 0
            dec0 = 0
            dec0_ps = 0
            for i in range(n):
                hdu = fits.open(path.join(cur_path,filenames[i]))
                cur_ra.append(hdu[0].header['RA'])
                cur_dec.append(hdu[0].header['DEC'])
                hdu.close()
                try:
                    ps = platesolve.platesolve(path.join(cur_path,filenames[i]),simulate.plate_scale_arcsec_px.value)
                    cur_ra_ps.append(ps['ra_j2000_hours'] * 15)
                    cur_dec_ps.append(ps['dec_j2000_degrees'])
                except:
                    cur_ra_ps.append(np.nan)
                    cur_dec_ps.append(np.nan)
                    
                if filenames[i] == 'cntr.fits':
                    ra0 = cur_ra[-1]
                    dec0 = cur_dec[-1]
                    ra0_ps = cur_ra_ps[-1]
                    dec0_ps = cur_dec_ps[-1]
            ra_vec.append(ra0)
            dec_vec.append(dec0)
            ra_shift_vec.append(np.array(cur_ra) - ra0)
            ra_corr_vec.append(np.array(cur_ra_ps) - ra0_ps)
            dec_shift_vec.append(np.array(cur_dec) - dec0)
            dec_corr_vec.append(np.array(cur_dec_ps) - dec0_ps)
            pbar.update(1)
    
    return ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec

def precision_to_csv(filename,ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec):
    ra_col = []
    dec_col = []
    ra_shift_col = []
    dec_shift_col = []
    ra_corr_col = []
    dec_corr_col = []

    for i in range(len(ra_corr_vec)):
        ra0 = ra_vec[i]
        dec0 = dec_vec[i]
        for j in range(len(ra_corr_vec[i])):
            ra_col.append(ra0)
            dec_col.append(dec0)
            ra_shift_col.append(ra_shift_vec[i][j])
            dec_shift_col.append(dec_shift_vec[i][j])
            ra_corr_col.append(ra_corr_vec[i][j])
            dec_corr_col.append(dec_corr_vec[i][j])

    df = pd.DataFrame({'ra':ra_col,'dec':dec_col,'ra_shift':ra_shift_col,
                    'dec_shift':dec_shift_col,'ra_corr':ra_corr_col,'dec_corr':dec_corr_col})
    filepath = path.join('.','ps_results',filename)
    df.to_csv(filepath,index=False)

def plot_guiding(ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec):
    fig,(ax1,ax2) = plt.subplots(1,2,figsize = (10,5),dpi=120)
    
    x1_vec = dec_shift_vec / (simulate.plate_scale_arcsec_px.value/3600)
    y1_vec = dec_corr_vec / (simulate.plate_scale_arcsec_px.value/3600)

    for x,y in zip(x1_vec,y1_vec):
        ax1.scatter(x[:10],y[:10],marker='x')
    ax1.plot(np.unique(x1_vec),np.unique(x1_vec),ls='--',color='k')
    ax1.set_yticks(np.arange(-4,4.5,0.5))
    ax1.set_xlabel('Actual shift (px)',fontdict={'size':14})
    ax1.set_ylabel('Correction (px)',fontdict={'size':14})
    ax1.set_title('plate solving for DEC correction')
    ax1.grid()
    ax1.set_ylim(-4,4)

    x2_vec = ra_shift_vec / (simulate.plate_scale_arcsec_px.value/3600)
    y2_vec = ra_corr_vec / (simulate.plate_scale_arcsec_px.value/3600)

    for x,y in zip(x2_vec,y2_vec):
        ax2.scatter(x,y,marker='x')
    ax2.plot(np.unique(x2_vec),np.unique(x2_vec),ls='--',color='k')
    ax2.set_yticks(np.arange(-4,4.5,0.5))
    ax2.set_xlabel('Actual shift (px)',fontdict={'size':14})
    ax2.set_ylabel('Correction (px)',fontdict={'size':14})
    ax2.set_title('plate solving for RA correction')
    ax2.grid()
    ax2.set_ylim(-4,4)

    plt.show()

def show_image(filepath):
    image = fits.open(path.normpath(filepath))
    plt.imshow(image[0].data,norm=LogNorm(),cmap='Greys',origin='lower')
    print(f'RA {image[0].header["RA"]}')
    print(f'DEC {image[0].header["DEC"]}')
    print(f'seeing {image[0].header["seeing"]}')
    print(f'bgd {image[0].header["bgdmagg"]}')
    image.close()
    
def get_error_arrays(dataframe_lst):
    err_lst = []
    for df in dataframe_lst:
        df = df.iloc[1::2]
        dec_err_arcsec = (df['dec_corr'] - df['dec_shift']) * 3600
        ra_err_arcsec = (df['ra_corr'] - df['ra_shift']) * 3600
        err_arr = (dec_err_arcsec**2 + (ra_err_arcsec * np.cos(np.deg2rad(df['dec'])))**2 )**(1/2)
        err_lst.append(err_arr)
    return err_lst

def plot_error_hist(err_lst,color_lst = ['LimeGreen','Crimson','RoyalBlue'],label_lst=['bgd 18.5/seeing 1','bgd 19.5/seeing 2','bgd 20.5/seeing 3']):
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_xlabel('correction error (arcsec)')
    txt_height = [0.95 - 0.2*i for i in range(len(err_lst))]

    for err_arr,clr,lbl,hgt in zip(err_lst,color_lst,label_lst,txt_height):
        ax.hist(err_arr,bins=100,range=(-3,3),histtype='step',label=lbl,color=clr)
        txt_str = '\n'.join((
        f"50% < {scoreatpercentile(np.abs(err_arr),50):1.2f}'' ",
        f"90% < {scoreatpercentile(np.abs(err_arr),90):1.2f}'' ",
        f"99% < {scoreatpercentile(np.abs(err_arr),99):1.2f}'' "))
        props = dict(boxstyle='round', facecolor=clr, alpha=0.5)
        ax.text(0.05, hgt, txt_str, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', bbox=props)
    ax.legend()
    plt.show()

In [5]:
ra = 92.1321189755064
dec = 11.691001657
bgd = 20.5
seeing = 1 * u.arcsec
texp = 1 * u.s
mag = 15
resolution = 1

make_image(ra,dec,bgd,seeing.value,texp.value,mag,resolution,'test','cntr')

In [6]:
show_image('./images/test/cntr.fits')

RA 92.1321189755064
DEC 11.691001657
seeing 1.0
bgd 20.5


In [3]:
# create_precision_images(30,20,1,4)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [29:27<00:00, 58.92s/it]


In [4]:
# ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec = precision_solve('./images/prec')

100%|██████████| 30/30 [03:57<00:00,  7.92s/it]


In [5]:
# plot_guiding(ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec)

In [5]:
# df1 = pd.read_csv('./ps_results/guiding_exp1_bgd18.5_seeing1.csv')
# df2 = pd.read_csv('./ps_results/guiding_exp1_bgd19.5_seeing2.csv')
# df3 = pd.read_csv('./ps_results/guiding_exp1_bgd20.5_seeing3.csv')

# err_lst = get_error_arrays([df1,df2,df3])
# plot_error_hist(err_lst)

In [6]:
# import astropy.units as u
# ra = 92.1321189755064
# dec = 11.691001657
# bgd = 20.5
# seeing = 1
# texp = 1
# mag = 15
# resolution = 4 
# make_image(ra,dec,bgd,seeing,texp,mag,resolution,'test','tst')
# show_image('./images/test/tst.fits')

In [7]:
# create_precision_images(300,20.5,3)
# ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec = precision_solve('./images/prec')
# precision_to_csv('guiding_exp1_bgd20.5_seeing3.csv',ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec)

In [8]:
# df1 = pd.read_csv('./ps_results/guiding_exp1_bgd18.5_seeing1.csv')
# df2 = pd.read_csv('./ps_results/guiding_exp1_bgd19.5_seeing2.csv')
# df3 = pd.read_csv('./ps_results/guiding_exp1_bgd20.5_seeing3.csv')